In [1]:
import json
from PIL import Image, ImageDraw, ImageFont
import os, shutil
import numpy as np
import pandas as pd
from matplotlib import colors, cm, pyplot as plt
import glob
import sys

In [2]:
PIXELS_X = 910
PIXELS_Y = 910  # equal to the number of scan lines

In [3]:
predictions_file = '/Users/darylwilding-mcbride/Downloads/experiments/dwm-test/predictions/yolov3-tiny_3l-tile-35-threshold-0_01-results.json'

In [4]:
tiles_dir = '/Users/darylwilding-mcbride/Downloads/experiments/dwm-test/tiles/190719_Hela_Ecoli_1to1_01/tile-35'
tiles_with_predictions_dir = '{}/predictions'.format(tiles_dir)
tiles_with_tracked_objects_dir = '{}/tracked-features'.format(tiles_dir)

In [5]:
if os.path.exists(tiles_with_predictions_dir):
    shutil.rmtree(tiles_with_predictions_dir)
os.makedirs(tiles_with_predictions_dir)

In [6]:
if os.path.exists(tiles_with_tracked_objects_dir):
    shutil.rmtree(tiles_with_tracked_objects_dir)
os.makedirs(tiles_with_tracked_objects_dir)

In [7]:
with open(predictions_file) as file:
    predictions = json.load(file)

In [8]:
feature_label_font = ImageFont.truetype('/Library/Fonts/Arial.ttf', 10)

In [10]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]

In [12]:
def iou(bb_test, bb_gt):
    # Computes IUO between two bounding boxes in the form [x1,y1,x2,y2]
    # Source: https://github.com/abewley/sort
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb_test[2]-bb_test[0])*(bb_test[3]-bb_test[1]) + (bb_gt[2]-bb_gt[0])*(bb_gt[3]-bb_gt[1]) - wh)
    return(o)


In [ ]:
MINIMUM_IOU_FOR_ASSOCIATION = 0.4

In [ ]:
detected_objects = []
feature_id = 0
for frame in predictions:
    prediction_frame_id = frame['frame_id']
    tile_name = os.path.basename(frame['filename'])
    source_tile_name = '{}/{}'.format(tiles_dir, tile_name)
    predictions_tile_name = '{}/{}'.format(tiles_with_predictions_dir, tile_name)
    tracked_objects_tile_name = '{}/{}'.format(tiles_with_tracked_objects_dir, tile_name)
    
    # load the source tile
    tile_predictions_img = Image.open(source_tile_name)
    draw_prediction = ImageDraw.Draw(tile_predictions_img)
    
    object_id = 1
    for prediction in frame['objects']:
        class_id = prediction['class_id']
        class_name = prediction['name']
        bounding_box = prediction['relative_coordinates']
        confidence = prediction['confidence']
        
        # calculate the coordinates
        x1 = (bounding_box['center_x'] - (bounding_box['width'] / 2)) * PIXELS_X
        y1 = (bounding_box['center_y'] - (bounding_box['height'] / 2)) * PIXELS_Y
        w = bounding_box['width'] * PIXELS_X
        h = bounding_box['height'] * PIXELS_Y
        x2 = x1 + w
        y2 = y1 + h

        # draw the prediction
        draw_prediction.rectangle(xy=[(x1, y1), (x2, y2)], fill=None, outline=(100,255,100,20))
        draw_prediction.text((x1, y1-12), '{} ({})'.format(class_name, round(confidence,1)), font=feature_label_font, fill=(100,255,100,20))
        
        # add this prediction to the list of objects to track
        detected_objects.append((prediction_frame_id, object_id, class_id, x1, y1, x2, y2, confidence, feature_id))
        
        object_id += 1

    # save the tile with the predictions overlaid
    tile_predictions_img.save(predictions_tile_name)

In [25]:
detected_objects_df = pd.DataFrame(detected_objects, columns=['prediction_frame_id','object_id','class_id','x1','y1','x2','y2','confidence','feature_id'])

In [26]:
detected_objects_df.head()

,prediction_frame_id,object_id,x1,y1,x2,y2,confidence,feature_id
0,1,1,678.166125,625.382485,806.400595,671.824335,1.000000,0
1,1,2,429.943605,655.831540,487.733155,703.662960,0.999964,0
2,1,3,822.008915,614.897465,919.814805,664.143935,0.999950,0
3,1,4,562.693950,481.818155,632.194290,508.340105,0.999898,0
4,1,5,138.911500,608.286770,277.919460,693.479150,0.905883,0


In [27]:
objects_frame_2 = detected_objects_df[detected_objects_df.prediction_frame_id == 2]
objects_frame_1 = detected_objects_df[detected_objects_df.prediction_frame_id == 1]

In [28]:
len(objects_frame_1), len(objects_frame_2)

(9, 8)

In [11]:
    # track the objects
    track_ids = update_feature_list(objects_in_this_frame)
    track_ids_l = list(map(tuple, track_ids))

    # draw the tracked objects
    for track_id in track_ids_l:
        x1 = int(track_id[0])
        y1 = int(track_id[1])
        x2 = int(track_id[2])
        y2 = int(track_id[3])
        feature_id = int(track_id[4])
        feature_colour = tableau20[feature_id % len(tableau20)]
        draw_tracked_objects.rectangle(xy=[(x1, y1), (x2, y2)], fill=None, outline=feature_colour)
        draw_tracked_objects.text((x1, y1-12), "feature {}".format(feature_id), font=feature_label_font, fill=feature_colour)

    # save the tile with the predictions overlaid
    tile_tracked_img.save(tracked_objects_tile_name)
    

In [23]:
predictions[0]

{'frame_id': 1,
 'filename': 'data/peptides/tile-35/frame-1889-tile-35-mz-730-748.png',
 'objects': [{'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.815696,
    'center_y': 0.712751,
    'width': 0.140917,
    'height': 0.051035},
   'confidence': 1.0},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.504218,
    'center_y': 0.746975,
    'width': 0.063505,
    'height': 0.052562},
   'confidence': 0.999964},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.957046,
    'center_y': 0.70277,
    'width': 0.107479,
    'height': 0.054117},
   'confidence': 0.99995},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.656532,
    'center_y': 0.544043,
    'width': 0.076374,
    'height': 0.029145},
   'confidence': 0.999898},
  {'class_id': 0,
   'name': 'charge-2',
   'relative_coordinates': {'center_x': 0.229028,
    'center_y': 0.715256,
    'width': 0.1527